In [2]:
# Src: ### Author: [Marco Tavora](http://www.marcotavora.me/)

In [22]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # see the value of multiple statements at once.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [26]:
#!python -m spacy download en_core_web_sm

In [27]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()

In [28]:
!ls

0 - EDA.ipynb
1 - Analysis.ipynb
ampreviews_scraped_forum_dataframe.joblib
ampreviews_text.csv
bs4_example_pt2.ipynb
list_urls.txt
notes_on_ampreviews_structure.md
topic-modeling-lda.ipynb


In [29]:
df = pd.read_csv('ampreviews_text.csv')
#df.columns = ['titles']
#df.shape
#df.head()
df.sample(n=2)

,msg_text
50,\nDid Hibiki move over to JPK? or back to Japa...
43,\nTitle: Review: Hibiki- Pleasure Palace\nDate...


In [30]:
doc_set = df.values.T.tolist()[0]
print(doc_set[0:10])

['\nIs hard to narrow down to "the one" , I do find myself booking the same girls every time...\nLKS - Jennie, but she only do doubles on weekends now.\nLSC - Izzy, but her looks will take a few points off.\nBGC - Kimmy, but she will be gone in 3 weeks.\n\xa0\n', '\nI will take Joah, Binna and Irene over any girls in those orgs\n\xa0\n', '\nMila, Adela, Daya at this time\n\xa0\n', '\nSeri, soda, Irene\n\xa0\n', '\nLKS - Adel\nLSC - Rose\nBGC - Margo\n\xa0\n', '\nDamn some dudes are lucky enough to be verified for all these 3 orgs!\n\xa0\n', '\nProbably old schoolers that got in before the all the new stricter verif process\n\xa0\n', '\nLKS - Jennie (that foursome with Sarah and Niay, wow, I need to make that happen)\nLSC - Sunny\nBGC - Daya (ymmv but hottest out there for sure)\n\xa0\n', "I'm new here.  May I get link for all 3 orgs and try my luck to get verified?  TIA", '\nGo here for links\nBayareakoreangirls.com\n\xa0\n']


In [31]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer 
from nltk.stem.porter import PorterStemmer

In [32]:
#stemmer = PorterStemmer()
stemmer = SnowballStemmer(language='english')
lemmatizer = WordNetLemmatizer()

In [33]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

### Build a list of lists of tokens 

In [34]:
tokenined_docs = []
for doc in doc_set:
    tokens = tokenizer.tokenize(doc.lower())
    tokenined_docs.append(tokens)
    
print(tokenined_docs[0:3])

[['is', 'hard', 'to', 'narrow', 'down', 'to', 'the', 'one', 'i', 'do', 'find', 'myself', 'booking', 'the', 'same', 'girls', 'every', 'time', 'lks', 'jennie', 'but', 'she', 'only', 'do', 'doubles', 'on', 'weekends', 'now', 'lsc', 'izzy', 'but', 'her', 'looks', 'will', 'take', 'a', 'few', 'points', 'off', 'bgc', 'kimmy', 'but', 'she', 'will', 'be', 'gone', 'in', '3', 'weeks'], ['i', 'will', 'take', 'joah', 'binna', 'and', 'irene', 'over', 'any', 'girls', 'in', 'those', 'orgs'], ['mila', 'adela', 'daya', 'at', 'this', 'time']]


In [35]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /Users/owls/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

[nltk_data] Downloading package omw-1.4 to /Users/owls/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Apply lemmatizing

In [36]:
lemmatized_tokens = []
for lst in tokenined_docs:
    tokens_lemma = [lemmatizer.lemmatize(i) for i in lst]
    lemmatized_tokens.append(tokens_lemma)
    
print(lemmatized_tokens[0:3])

[['is', 'hard', 'to', 'narrow', 'down', 'to', 'the', 'one', 'i', 'do', 'find', 'myself', 'booking', 'the', 'same', 'girl', 'every', 'time', 'lks', 'jennie', 'but', 'she', 'only', 'do', 'double', 'on', 'weekend', 'now', 'lsc', 'izzy', 'but', 'her', 'look', 'will', 'take', 'a', 'few', 'point', 'off', 'bgc', 'kimmy', 'but', 'she', 'will', 'be', 'gone', 'in', '3', 'week'], ['i', 'will', 'take', 'joah', 'binna', 'and', 'irene', 'over', 'any', 'girl', 'in', 'those', 'orgs'], ['mila', 'adela', 'daya', 'at', 'this', 'time']]


### Dropping stopwords and words with less than $n$ letters

In [37]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/owls/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
from nltk.corpus import stopwords
en_stop_words = stopwords.words('english')


In [39]:
n=3
tokens = []
for lst in lemmatized_tokens:
    tokens.append([i for i in lst if not i in en_stop_words if len(i) > n])

print(tokens[0:3])

[['hard', 'narrow', 'find', 'booking', 'girl', 'every', 'time', 'jennie', 'double', 'weekend', 'izzy', 'look', 'take', 'point', 'kimmy', 'gone', 'week'], ['take', 'joah', 'binna', 'irene', 'girl', 'orgs'], ['mila', 'adela', 'daya', 'time']]


## Document-term matrix

[[go back to the top]](#Table-of-contents)

I will now generate an LDA model and for that, the frequency that each term occurs within each document needs to be understood.

A **document-term matrix** is constructed to do that. It contains a corpus of $n$ documents and a vocabulary of $m$ words. Each cell $ij$ counts the frequency of the word $j$ in the document $i$.

## Tokens into dictionary

[[go back to the top]](#Table-of-contents)

In [40]:
from gensim import corpora, models

dictionary = corpora.Dictionary(tokens)

## Tokenize documents into document-term matrix

[[go back to the top]](#Table-of-contents)

In [41]:
corpus = [dictionary.doc2bow(text) for text in tokens]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [42]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1)]

## LDA model

## LDA model

In [43]:
import gensim
ldamodel_3 = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)
ldamodel_3.save('model3.gensim')
ldamodel_4 = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)
ldamodel_4.save('model4.gensim')

In [44]:
for el in ldamodel_3.print_topics(num_topics=3, num_words=3):
    print(el,'\n')

(0, '0.019*"back" + 0.019*"girl" + 0.014*"great"') 

(1, '0.027*"time" + 0.021*"would" + 0.018*"photo"') 

(2, '0.030*"review" + 0.015*"city" + 0.015*"photo"') 



In [45]:
for el in ldamodel_4.print_topics(num_topics=3, num_words=3):
    print(el,'\n')

(0, '0.026*"girl" + 0.023*"time" + 0.014*"back"') 

(1, '0.026*"kaori" + 0.013*"look" + 0.013*"good"') 

(2, '0.028*"review" + 0.027*"photo" + 0.016*"city"') 



In [46]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')

In [47]:
corpus = pickle.load(open('corpus.pkl', 'rb'))

In [48]:
lda = gensim.models.ldamodel.LdaModel.load('model3.gensim')

In [49]:
import pyLDAvis.gensim_models


/Users/owls/v3/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [50]:
lda_display = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary, sort_topics=False)

/Users/owls/v3/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/Users/owls/v3/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/owls/v3/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/owls/v3/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alte

In [51]:
pyLDAvis.display(lda_display)